In [1]:
import numpy as np
import csv
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold
from scipy import interp
from  matplotlib import pyplot as plt
from sklearn import cross_validation
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn import metrics, preprocessing
import pickle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [4]:
##drop passengerID and Name as these will not be much help in classification
data = data.drop( [ "PassengerId",'Name','Ticket','Cabin'],axis=1)

In [5]:
## fill na values with the column mean
data = data.fillna(data.mean())


In [6]:
data.tail(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
886,0,2,male,27.000000,0,0,13.00,S
887,1,1,female,19.000000,0,0,30.00,S
888,0,3,female,29.699118,1,2,23.45,S
889,1,1,male,26.000000,0,0,30.00,C
890,0,3,male,32.000000,0,0,7.75,Q


In [7]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22,1,0,7.2500,S
1,1,1,female,38,1,0,71.2833,C
2,1,3,female,26,0,0,7.9250,S
3,1,1,female,35,1,0,53.1000,S
4,0,3,male,35,0,0,8.0500,S


In [8]:
data = data.fillna(data.mode())

In [9]:
data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [10]:
## one hot encode all of the categorical values Pclass, Embarked, Sex
## code example for one hot encoding : http://corpocrat.com/2014/08/29/tutorial-titanic-dataset-machine-learning-for-kaggle/
dummy_values = []
cols = ['Pclass','Sex','Embarked']
for col in cols:
    dummy_values.append(pd.get_dummies(data[col]))

In [11]:
one_hot_values = pd.concat(dummy_values, axis=1)

In [12]:
data = pd.concat((data,one_hot_values),axis=1)

In [13]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,1,2,3,female,male,C,Q,S
0,0,3,male,22,1,0,7.2500,S,0,0,1,0,1,0,0,1
1,1,1,female,38,1,0,71.2833,C,1,0,0,1,0,1,0,0
2,1,3,female,26,0,0,7.9250,S,0,0,1,1,0,0,0,1
3,1,1,female,35,1,0,53.1000,S,1,0,0,1,0,0,0,1
4,0,3,male,35,0,0,8.0500,S,0,0,1,0,1,0,0,1


In [14]:
## drop the non-one-hot encoded values 
# data = data.drop( ['Sex', 'Embarked'],axis=1)
data = data.drop( ['Sex', 'Embarked', 'Pclass' ],axis=1)

In [15]:
data.head(5)

,Survived,Age,SibSp,Parch,Fare,1,2,3,female,male,C,Q,S
0,0,22,1,0,7.2500,0,0,1,0,1,0,0,1
1,1,38,1,0,71.2833,1,0,0,1,0,1,0,0
2,1,26,0,0,7.9250,0,0,1,1,0,0,0,1
3,1,35,1,0,53.1000,1,0,0,1,0,0,0,1
4,0,35,0,0,8.0500,0,0,1,0,1,0,0,1


In [16]:
data.describe()

,Survived,Age,SibSp,Parch,Fare,1,2,3,female,male,C,Q,S
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,29.699118,0.523008,0.381594,32.204208,0.242424,0.206510,0.551066,0.352413,0.647587,0.188552,0.086420,0.722783
std,0.486592,13.002015,1.102743,0.806057,49.693429,0.428790,0.405028,0.497665,0.477990,0.477990,0.391372,0.281141,0.447876
min,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,29.699118,0.000000,0.000000,14.454200,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,1.000000,35.000000,1.000000,0.000000,31.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,1.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
y = data['Survived'].as_matrix()

In [18]:
X =data.iloc[:,1:].values

In [19]:
print(X[0:2])

[[ 22.       1.       0.       7.25     0.       0.       1.       0.       1.
    0.       0.       1.    ]
 [ 38.       1.       0.      71.2833   1.       0.       0.       1.       0.
    1.       0.       0.    ]]


In [20]:
print ("X shape is: ", X.shape, "Y shape is" , y.shape, )

X shape is:  (891, 12) Y shape is (891,)


In [21]:
##make sure there are no nan values
np.any(np.isnan(X)), np.any(np.isnan(y))

(False, False)

In [22]:
## from Mike's code along http://nbviewer.jupyter.org/gist/gcr/48a6593835044e92c780fc1922df0492
pre = preprocessing.StandardScaler()
pre.fit(X)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [23]:
pre.transform(X)

array([[-0.5924806 ,  0.43279337, -0.47367361, ..., -0.48204268,
        -0.30756234,  0.61930636],
       [ 0.63878901,  0.43279337, -0.47367361, ...,  2.0745051 ,
        -0.30756234, -1.61470971],
       [-0.2846632 , -0.4745452 , -0.47367361, ..., -0.48204268,
        -0.30756234,  0.61930636],
       ..., 
       [ 0.        ,  0.43279337,  2.00893337, ..., -0.48204268,
        -0.30756234,  0.61930636],
       [-0.2846632 , -0.4745452 , -0.47367361, ...,  2.0745051 ,
        -0.30756234, -1.61470971],
       [ 0.17706291, -0.4745452 , -0.47367361, ..., -0.48204268,
         3.25137334, -1.61470971]])

In [24]:
## Divide data into train and test
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X, y, test_size=0.25, random_state=42)

In [25]:
clf = LogisticRegression(C=1)
clf.fit((X_train),y_train)
clf.score((X_test),y_test)

0.7982062780269058

In [26]:
## from Mike's code along http://nbviewer.jupyter.org/gist/gcr/48a6593835044e92c780fc1922df0492
## find best C
def evaluate(C):
    return LogisticRegression(C=C) \
        .fit(pre.fit_transform(X_train), y_train) \
        .score(pre.transform(X_test), y_test)

In [27]:
def bestModel(allCs):
    """ Iterates through all of the C values we pass, and returns a tuple of the score of the best C,
    and the best C"""
    score = 0
    c_val = .001
    for c in allCs:
        currScore = evaluate(c)
        if currScore > score:
            score = currScore
            c_val = c
    return score, c_val

In [28]:
## We see that the best performing C is 1, which I have used above
possible_Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
bestC = bestModel(possible_Cs)
print(bestC)

(0.80269058295964124, 1)


In [29]:
##gives a realistic depiction of how the model will perform using cross validation
scores = cross_validation.cross_val_score(clf, X, y, cv=5)
scores.mean()

0.79577340416137821

In [30]:
##uses K fold to gather scores for each fold
def evaluate(fold):
    # Input: A tuple of (train, test) indices
    train, test = fold
    return LogisticRegression() \
        .fit(X[train], y[train]) \
        .score(X[test], y[test])

all_folds = cross_validation.KFold(len(X), n_folds=10)

In [31]:
## mean scores are 
scores = []
for fold in all_folds:
    scores.append(evaluate(fold))
mean(scores)

0.8002496878901374

In [33]:
clf_full = LogisticRegression(C=1)
clf_full.fit(X , y)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [34]:
## save model to load later
from sklearn.externals import joblib
joblib.dump(clf_full, 'log_reg_full.pkl') 

['log_reg_full.pkl',
 'log_reg_full.pkl_01.npy',
 'log_reg_full.pkl_02.npy',
 'log_reg_full.pkl_03.npy']

In [35]:
## Load model
clf2_full = joblib.load('log_reg_full.pkl') 

In [36]:
clf2_full.predict(X[0:1])

array([0])